In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08603954315185547
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 29.442078192502088
200000 47.70422580017152
300000 53.6170802375668
400000 53.66652281310571
500000 53.048581374271826
600000 54.76064001922731
700000 53.77107726651113
800000 57.39012490056375
900000 67.33255665589886
1000000 76.85880489128277
1100000 70.32348045163738
1200000 78.98377431984026
1300000 26.304235541595112
1400000 82.79847300435452
1500000 95.14038952502135
1600000 95.16434961846885
1700000 93.92509223350581
1800000 94.78938554496217
1900000 100.61865907716702
2000000 104.24143384259008
2100000 103.91572345875991
2200000 103.60834648701234
2300000 98.98106356650655
2400000 102.0009033245242
2500000 93.39957504711865
2600000 32.278580444498864
2700000 103.03221690263966
2800000 99.89486144946885
2900000 102.59972279190937
3000000 97.116340

29400000 105.75914432974136
29500000 102.39961979959895
29600000 104.87297111699145
29700000 108.10551450086122
29800000 106.42983598056428
29900000 109.9333313015361
30000000 69.47531011264743
30100000 85.70425967230857
30200000 114.1132932192975
30300000 105.62536676274722
30400000 87.26092420871547
30500000 55.49399494226062
30600000 106.89580076547675
30700000 105.16679047280822
30800000 104.40650696522405
30900000 102.35155407146398
31000000 106.49353572728107
31100000 104.6660663104412
31200000 101.34360012846241
31300000 100.18206070293878
31400000 100.74678532490199
31500000 18.421577778124288
31600000 104.49603310320384
31700000 99.01738435786244
31800000 99.02230567266655
31900000 101.80053073357819
32000000 105.38470293875712
32100000 101.52291582122534
32200000 104.90572475851582
32300000 108.5481739746625
32400000 102.86090817358769
32500000 62.35143573786278
32600000 83.24317343010485
32700000 115.77204157024633
32800000 111.53217310436791
32900000 115.9529054447858
33000

59200000 44.580885519752556
59300000 96.27313877670828
59400000 100.65458131902346
59500000 100.24842518151365
59600000 101.28385328196885
59700000 102.80634773560281
59800000 100.83137440049012
59900000 108.63320701966505
60000000 35.50391131984611
60100000 117.85307752085674
60200000 106.77384821750655
60300000 106.60701771824564
60400000 107.42449114796311
60500000 101.99090884531455
60600000 103.78204603686827
60700000 107.40822607349409
60800000 88.5250816497344
60900000 52.4582325303161
61000000 97.54770500715222
61100000 97.95317355077782
61200000 96.93065263605929
61300000 103.93662653745416
61400000 92.26955222175125
61500000 91.6540099817666
61600000 87.7464299952645
61700000 28.249439447667505
61800000 89.43869347969124
61900000 97.56900676318295
62000000 93.90861295706529
62100000 94.5321300434512
62200000 95.85120452640324
62300000 93.98385676062512
62400000 99.65933218452197
62500000 33.82835025476587
62600000 104.36990475177133
62700000 100.18996741124943
62800000 104.01

89300000 67.01883432559106
89400000 54.01135851251324
89500000 91.19152484549461
89600000 90.41234744838249
89700000 84.39973944332868
89800000 90.10172011299676
89900000 73.8666037634217
90000000 62.21488582157368
90100000 94.71603297258497
90200000 97.89776968556153
90300000 99.02121022789487
90400000 95.96491363052796
90500000 104.80714396863678
90600000 91.8830632635487
90700000 95.3313223744788
90800000 95.66997010212566
90900000 98.24562771707487
91000000 93.05311305177563
91100000 91.26028636187826
91200000 80.39835148186674
91300000 39.65929647052689
91400000 87.75533621589287
91500000 87.60995482552971
91600000 92.70479003504661
91700000 85.84356962213413
91800000 73.48881088850072
91900000 39.45147639548902
92000000 88.98820802880454
92100000 90.15058554118133
92200000 88.67412339462578
92300000 88.02912513824016
92400000 68.77665930614315
92500000 58.79670263978408
92600000 95.75360263869295
92700000 90.36997554610032
92800000 99.77669968908647
92900000 97.07094623630059
930

119000000 90.4053627463814
119100000 62.465963705185736
119200000 57.00765797453353
119300000 87.26211160216987
119400000 96.53012221186619
119500000 53.459068636897115
119600000 71.47264603854853
119700000 85.18832986768263
119800000 96.13027016902879
119900000 51.94582588200305
120000000 76.13086466245382
120100000 90.90176084273804
120200000 93.66552701812111
120300000 90.57283221121226
120400000 98.03330735763414
120500000 94.66465462956857
120600000 100.46295022958586
120700000 105.25447860902506
120800000 101.68650742891062
120900000 99.97750511428666
121000000 97.0311280111772
121100000 95.85387208357747
121200000 98.28967470979444
121300000 93.30677859720365
121400000 96.26445506087455
121500000 91.80151375183418
121600000 90.30817150421571
121700000 30.831602398625733
121800000 89.83187047085549
121900000 91.91911906302923
122000000 64.13776224044167
122100000 56.05644886722751
122200000 84.93957154619048
122300000 87.29553945249874
122400000 45.86415675296996
122500000 75.388

...

953100000 1.6437425580261449
953200000 1.4424430728321738
953300000 1.485341032515627
953400000 0.7420228654491128
953500000 0.947805258068223
953600000 1.685723184283546
953700000 2.4072178928870698
953800000 1.842430533740127
953900000 1.4850061935307595
954000000 1.0715431521613559
954100000 0.6009804570594602
954200000 1.601705261853793
954300000 1.8033596810693442
954400000 1.6328474457152888
954500000 1.4939799756823693
954600000 1.3441097774365582
954700000 0.5227216785023384
954800000 1.4173100361188067
954900000 1.6047641941696404
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.7858286196177482
955100000 1.6543876867623812
955200000 1.392434859059632
955300000 1.2444548397959148
955400000 1.2150857831112793
955500000 1.3189847753819028
955600000 1.162932803081945
955700000 1.5683513892517449
955800000 1.4613882842638772
955900000 1.149876303925134
956000000 1.8754971306498212
956100000 1.2855038652196527
956200000 1.2160611940338069
956300000 

979200000 0.31771394867702524
979300000 0.3497933974009216
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.318175106787618
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3067748539505303
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.6499234582188185
979700000 2.1974115227046203
979800000 1.7077791031664153
979900000 1.3254624758680231
980000000 1.377356235003459
980100000 2.2934520555981557
980200000 1.453880134899556
980300000 1.4310045510658709
980400000 1.2571567850190282
980500000 1.4195135401719492
980600000 1.257867126388581
980700000 1.321664873035713
980800000 1.7612275340922432
980900000 1.1905155226030164
981000000 1.5272990821407257
981100000 1.6049767340988814
981200000 1.063

998800000 1.1136880221740675
998900000 1.2800360768817338
999000000 0.8637989999004663
999100000 1.074149478123394
999200000 1.2863321782252637
999300000 0.997403689249748
999400000 0.9821007147852135
999500000 1.2769239935596572
999600000 1.0509628689280153
999700000 0.8131513204666806
999800000 1.2452406242206402
999900000 1.244775304070435
1000000000 0.7683691267854644
1000100000 1.2947621057524759
1000200000 1.3108338224733531
1000300000 0.867533660970099
1000400000 1.3292417112149928
1000500000 0.24001060262884497
1000600000 0.2391969760626879
1000700000 0.23889206305207014
1000800000 0.2374096677806914
1000900000 0.23707826645452906
1001000000 0.2365938881909874
1001100000 0.23667051684243084
1001200000 0.2372475520287004
1001300000 0.23263358410153986
1001400000 0.23500068262141544
1001500000 0.2339788267660598
1001600000 0.2331855193315347
1001700000 0.26327710818557937
1001800000 0.232758185209175
1001900000 0.2307262118262344
1002000000 0.6317890419284701
1002100000 1.3640871

1025700000 0.2617482253387193
1025800000 0.2360815182147523
1025900000 0.3279240994753268
1026000000 0.28507389497470853
1026100000 0.20155603109947576
1026200000 0.3275191753890203
1026300000 0.2978168275997341
1026400000 0.16334961303125026
1026500000 0.339146173886629
1026600000 0.3005941562456012
1026700000 0.16838756092766125
1026800000 0.3353541358884434
1026900000 0.29608366922947765
1027000000 0.19428886477565432
1027100000 0.30360155456701715
1027200000 0.3039019597046435
1027300000 0.21565518554691543
1027400000 0.2775366115057488
1027500000 0.3180077633723577
1027600000 0.23601849964104957
1027700000 0.28112596958044966
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.31511435931827825
1027900000 0.25019605782227056
1028000000 0.23077290546490087
1028100000 0.25118193743596673
1028200000 0.25065793066272274
1028300000 0.22372898148411777
1028400000 0.23981786154671908
1028500000 0.25325534918917947
1028600000 0.22061860271123845
1028700000 0

1045400000 0.0424421369089769
1045500000 0.04949446796987852
1045600000 0.044360953815375435
1045700000 0.03958491107616093
1045800000 0.04371283732313316
1045900000 0.04315043393968543
1046000000 0.027158072714985076
1046100000 0.010169996532833576
1046200000 0.028491845630222556
1046300000 0.02863693327781757
1046400000 0.03177554921573202
1046500000 0.027014735383669535
1046600000 0.025472827940922316
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.026852985956082744
1046800000 0.014090201323989363
1046900000 0.013198980324104099
1047000000 0.010359200132101773
1047100000 0.0051494997434417406
1047200000 0.008239627549253569
1047300000 0.00890806945732236
best so far: 0
type: [4, 1, 1, 17, 2] 136
cases of this type: 78608
1047400000 0.005948145392413934
best so far: 0
type: [4, 1, 1, 34, 1] 136
cases of this type: 39304
best so far: 0
type: [4, 1, 2, 1, 17] 136
cases of this type: 334084
1047500000 0.0036739579800917047
1047600000 0.0052592349490106